In [95]:
from datetime import datetime
def parse_time(timestamp):
    """
    'YYYY-MM-DD HH:MM:SS AM/PM', 'YYYY-MM-DD HH:MM:SS', 또는 'YYYY-MM-DD HH:MM' 형식을 파싱하는 함수
    """
    timestamp = timestamp.split('.')[0]
    try:
        # 먼저 24시간 형식으로 파싱 시도
        return datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return datetime.strptime(timestamp, '%Y-%m-%d %I:%M:%S %p')
        except ValueError:
            # 실패하면 'YYYY-MM-DD HH:MM' 형식으로 파싱 시도
            return datetime.strptime(timestamp, '%Y-%m-%d %H:%M')

In [96]:
import pandas as pd
bus_df = pd.read_csv('test.csv')

In [97]:
print(bus_df.isnull().sum())

MASK_SELECTED        0
LAT                  0
LNG                  0
STOP_ID              0
Parsed_Date          0
up_down              0
temperature          0
Relative_Humidity    0
is_weekend           0
day_name             0
previous             0
wind_d               0
wind_s               0
Bus_num              0
prev_arrive_time     0
start_time           0
travel_time          0
dtype: int64


In [98]:
bus_df['timestamp'] = bus_df['Parsed_Date'].apply(parse_time)
bus_df.dtypes

MASK_SELECTED                 int64
LAT                         float64
LNG                         float64
STOP_ID                       int64
Parsed_Date                  object
up_down                       int64
temperature                 float64
Relative_Humidity           float64
is_weekend                     bool
day_name                     object
previous                    float64
wind_d                      float64
wind_s                      float64
Bus_num                       int64
prev_arrive_time             object
start_time                   object
travel_time                 float64
timestamp            datetime64[ns]
dtype: object

In [99]:
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# 데이터 전처리 함수
def preprocess_data(data, sequence_length=14, train_ratio=0.8, val_ratio=0.15):
    # 주기성 변환
    data["stop_sin"] = np.sin(2 * np.pi * data["MASK_SELECTED"] / sequence_length)
    data["stop_cos"] = np.cos(2 * np.pi * data["MASK_SELECTED"] / sequence_length)
    data["hour"] = data["timestamp"].dt.hour
    data["hour_sin"] = np.sin(2 * np.pi * data["hour"] / 24)
    data["hour_cos"] = np.cos(2 * np.pi * data["hour"] / 24)
    
    # 원핫 인코딩 적용
    data = pd.get_dummies(data, columns=['day_name', ])
   

    # 필요없는 열 제거 및 텐서 변환 준비
    features = data[[
        "MASK_SELECTED", "temperature", "LAT", "LNG", "Relative_Humidity", 'wind_d', 'wind_s', 'is_weekend',
        "previous", "stop_sin", "hour_sin", 'day_name_Monday', 'day_name_Saturday', 'day_name_Sunday',
       'day_name_Thursday', 'day_name_Tuesday', 'day_name_Wednesday'
    ]].values
    target = data["travel_time"].values.reshape(-1, 1)

    # 스케일링 적용
    scaler_features = MinMaxScaler(feature_range=(0, 1))
    scaler_target = MinMaxScaler(feature_range=(0, 1))
    features_scaled = scaler_features.fit_transform(features)  # timestamp 포함
    target_scaled = scaler_target.fit_transform(target)
    """
    # 데이터셋 분할 및 시퀀스 생성 함수
    num_sequences = len(features_scaled) // sequence_length
    train_size = int(num_sequences * train_ratio) * sequence_length
    val_size = int(num_sequences * val_ratio) * sequence_length
    
    X_train = features_scaled[:train_size]
    y_train = target_scaled[:train_size]
    
    X_val = features_scaled[train_size:train_size + val_size]
    y_val = target_scaled[train_size:train_size + val_size]
    
    X_test = features_scaled[train_size + val_size:]
    y_test = target_scaled[train_size + val_size:]
    """
    
    train_size = int(len(features_scaled) * 0.7)
    val_size = int(len(features_scaled) * 0.2)
    test_size = len(features_scaled) - train_size - val_size

    X_train, X_val, X_test = features_scaled[0:train_size,:], features_scaled[train_size:train_size+val_size,:], features_scaled[train_size+val_size:len(features_scaled),:]
    y_train, y_val, y_test = target_scaled[0:train_size,:], target_scaled[train_size:train_size+val_size,:], target_scaled[train_size+val_size:len(target_scaled),:]
        
    # 시퀀스 데이터 생성 함수
    def create_sequences(X, y, seq_length):
        sequences = []
        targets = []
        for i in range(0, len(X) - seq_length + 1, seq_length):
            sequences.append(X[i:i + seq_length])
            targets.append(y[i:i + seq_length])
        return torch.tensor(np.array(sequences), dtype=torch.float32), torch.tensor(np.array(targets), dtype=torch.float32)

    # 학습, 검증, 테스트 데이터셋 생성
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, sequence_length)
    X_val_seq, y_val_seq = create_sequences(X_val, y_val, sequence_length)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, sequence_length)
    
    return X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, scaler_target


In [100]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# DataLoader 생성
def create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test):
    
    X_train = torch.tensor(X_train).float()
    y_train = torch.tensor(y_train).float()
    X_val = torch.tensor(X_val).float()
    y_val = torch.tensor(y_val).float()
    X_test = torch.tensor(X_test).float()
    y_test = torch.tensor(y_test).float()

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    
    # batch_size=None 또는 데이터셋 크기만큼 설정
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    return train_loader, val_loader, test_loader


In [101]:
import torch
import torch.nn as nn

class Stacked_Lstm(nn.Module):
    def __init__(self, num_stops=28, input_size=15, hidden_size=16, dropout_prob=0.3):
        super(Stacked_Lstm, self).__init__()
        
        # LSTM layers
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=512, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=512, hidden_size=256, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
        self.lstm4 = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)
        
        # Layer Normalization
        self.layer_norm1 = nn.LayerNorm(512)
        self.layer_norm2 = nn.LayerNorm(256)
        
        # Dropout
        self.dropout = nn.Dropout(p=dropout_prob)
        
        # Fully Connected layers
        self.fc1 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_1, _ = self.lstm1(x)
        lstm_1 = self.layer_norm1(lstm_1)
        lstm_1 = self.dropout(lstm_1)
        
        lstm_2, _ = self.lstm2(lstm_1)
        lstm_2 = self.layer_norm2(lstm_2)
        lstm_2 = self.dropout(lstm_2)
        
        lstm_3, _ = self.lstm3(lstm_2)
        lstm_3 = self.dropout(lstm_3)
        
        lstm_4, _ = self.lstm4(lstm_3)
        lstm_4 = self.dropout(lstm_4)
        
        
        output = self.fc1(lstm_4)  # (batch_size, seq_len, 1)
        return output


In [102]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=50, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): 얼마나 많은 에폭 동안 검증 손실이 개선되지 않아도 학습을 계속할지.
            verbose (bool): True일 경우, 각 개선 사항을 출력.
            delta (float): 개선으로 간주하기 위한 최소 변화.
            path (str): 최상의 모델을 저장할 경로.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss  # 손실이 낮을수록 좋으므로 음수로 변환

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''검증 손실이 감소하면 모델을 저장'''
        if self.verbose:
            print("===================================saved===================================")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [103]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = bus_df
X_train, y_train, X_val, y_val, X_test, y_test, scaler_target = preprocess_data(data)

# DataLoader 생성
train_loader, val_loader, test_loader = create_dataloaders(X_train, y_train, X_val, y_val, X_test, y_test)

# Instantiate the model
model = Stacked_Lstm(input_size=17).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
early_stopping = EarlyStopping(patience=20, verbose=True, path='batch32/lstm_model2.pt')
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            output = model(
            batch_X
            )
            
            optimizer.zero_grad()
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                output = model(
                   batch_X
                )
                
                loss = criterion(output, batch_y)
                val_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")
        early_stopping(val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
# 학습 실행
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs=200)

/tmp/ipykernel_413797/4158508312.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).float()
/tmp/ipykernel_413797/4158508312.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).float()
/tmp/ipykernel_413797/4158508312.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val).float()
/tmp/ipykernel_413797/4158508312.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach()

Epoch 1/200, Train Loss: 0.05031997291371226, Val Loss: 0.034857381135225296
===================================saved===================================
Epoch 2/200, Train Loss: 0.036775270476937294, Val Loss: 0.04209090210497379
EarlyStopping counter: 1 out of 20
Epoch 3/200, Train Loss: 0.03952181036584079, Val Loss: 0.03382503613829613
===================================saved===================================
Epoch 4/200, Train Loss: 0.03774074371904135, Val Loss: 0.04104020446538925
EarlyStopping counter: 1 out of 20
Epoch 5/200, Train Loss: 0.036903990898281336, Val Loss: 0.03341168072074652
===================================saved===================================
Epoch 6/200, Train Loss: 0.03175481245853007, Val Loss: 0.03214003425091505
===================================saved===================================
Epoch 7/200, Train Loss: 0.031214430928230286, Val Loss: 0.029605451971292496
===================================saved===================================
Epoch 8/200, 

In [104]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
model.load_state_dict(torch.load('batch32/lstm_model2.pt'))
# 모델 평가 모드 설정
model.eval()

# 예측 결과와 실제값을 저장할 리스트 생성
predictions = []
actuals = []

# 예측 및 실제값 계산
with torch.no_grad():
    for batch in test_loader:
        # 배치에서 수치형 데이터와 타겟 값 분리
        batch_X, batch_y = batch[0].to(device), batch[1].to(device)

        # 모델 예측
        output = model(batch_X)  # output: (batch_size, seq_len=1, 1)

        # 출력값과 타겟값의 차원 맞추기
        output = output.squeeze(-1).squeeze(-1)  # (batch_size,)
        batch_y = batch_y.squeeze(-1)           # (batch_size,)

        # 예측값과 실제값을 리스트에 저장
        predictions.append(output.cpu().numpy())  # GPU에서 CPU로 이동 후 numpy 변환
        actuals.append(batch_y.cpu().numpy())     # GPU에서 CPU로 이동 후 numpy 변환

# 리스트를 하나의 배열로 결합
predictions = np.concatenate(predictions, axis=0)  # (전체 샘플 수,)
actuals = np.concatenate(actuals, axis=0)          # (전체 샘플 수,)

# 예측값과 실제값을 2차원 배열로 변환 (MinMaxScaler 사용을 위해)
predictions = predictions.reshape(-1, 1)
actuals = actuals.reshape(-1, 1)

# 예측값과 실제값의 스케일 복원
predictions_original = scaler_target.inverse_transform(predictions)
actuals_original = scaler_target.inverse_transform(actuals)

# 데이터프레임으로 저장
results_df = pd.DataFrame({
    "Actual": actuals_original.flatten(),
    "Predicted": predictions_original.flatten()
})

# CSV 파일로 저장
results_df.to_csv("test_predictions.csv", index=False)

# MSE 계산
mse = mean_squared_error(actuals_original, predictions_original)
rmse = np.sqrt(mse)

# 결과 출력
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print("예측 결과가 test_predictions.csv 파일로 저장되었습니다.")


Mean Squared Error (MSE): 1042.1383056640625
Root Mean Squared Error (RMSE): 32.28216552734375
예측 결과가 test_predictions.csv 파일로 저장되었습니다.


/tmp/ipykernel_413797/3933972543.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('batch32/lstm_model2.pt'))


Test1 don't touch it (Sequence O)

In [203]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
model.load_state_dict(torch.load('batch32/lstm_model2.pt'))
# 모델 평가 모드 설정
model.eval()

# 예측 결과와 실제값을 저장할 리스트 생성
predictions = []
actuals = []

# 예측 및 실제값 계산
with torch.no_grad():
    for batch in test_loader:
        # 배치에서 수치형 데이터와 타겟 값 분리
        batch_X, batch_y = batch[0].to(device), batch[1].to(device)

        # 모델 예측
        output = model(batch_X)  # output: (batch_size, seq_len=1, 1)

        # 출력값과 타겟값의 차원 맞추기
        output = output.squeeze(-1).squeeze(-1)  # (batch_size,)
        batch_y = batch_y.squeeze(-1)           # (batch_size,)

        # 예측값과 실제값을 리스트에 저장
        predictions.append(output.cpu().numpy())  # GPU에서 CPU로 이동 후 numpy 변환
        actuals.append(batch_y.cpu().numpy())     # GPU에서 CPU로 이동 후 numpy 변환

# 리스트를 하나의 배열로 결합
predictions = np.concatenate(predictions, axis=0)  # (전체 샘플 수,)
actuals = np.concatenate(actuals, axis=0)          # (전체 샘플 수,)

# 예측값과 실제값을 2차원 배열로 변환 (MinMaxScaler 사용을 위해)
predictions = predictions.reshape(-1, 1)
actuals = actuals.reshape(-1, 1)

# 예측값과 실제값의 스케일 복원
predictions_original = scaler_target.inverse_transform(predictions)
actuals_original = scaler_target.inverse_transform(actuals)

# 데이터프레임으로 저장
results_df = pd.DataFrame({
    "Actual": actuals_original.flatten(),
    "Predicted": predictions_original.flatten()
})

# CSV 파일로 저장
results_df.to_csv("test_predictions.csv", index=False)

# MSE 계산
mse = mean_squared_error(actuals_original, predictions_original)
rmse = np.sqrt(mse)

# 결과 출력
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print("예측 결과가 test_predictions.csv 파일로 저장되었습니다.")


Mean Squared Error (MSE): 1531.712646484375
Root Mean Squared Error (RMSE): 39.13710021972656
예측 결과가 test_predictions.csv 파일로 저장되었습니다.


/tmp/ipykernel_301035/3933972543.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('batch32/lstm_model2.pt'))
